In [5]:
from cerebralcortex.util.helper_methods import get_study_names
sn = get_study_names("/home/jupyter/cc3_conf/")
print(sn)
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, DoubleType,MapType, StringType,ArrayType, FloatType, TimestampType, IntegerType
from pyspark.sql.functions import minute, second, mean, window
from pyspark.sql import functions as F
import numpy as np
import pandas as pd
from cerebralcortex.core.datatypes import DataStream
from cerebralcortex.core.metadata_manager.stream.metadata import Metadata, DataDescriptor, \
ModuleMetadata
from typing import List
import numpy as np
from scipy import signal
import pandas as pd
from cerebralcortex import Kernel
from pyspark.sql import functions as F
CC = Kernel("/home/jupyter/cc3_conf/", study_name='rice')

KeyboardInterrupt: 

In [2]:
import zipfile,shutil

shutil.make_archive('../images_sampling/','zip','../images_sampling/')

In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, TimestampType, IntegerType
import numpy as np
from cerebralcortex.core.datatypes import DataStream
from cerebralcortex.core.metadata_manager.stream.metadata import Metadata, DataDescriptor, \
    ModuleMetadata
from scipy import signal
def filter_data(X,
                Fs=100,
                low_cutoff=.3,
                high_cutoff=5.0,
                filter_order=65):
    """
    Bandpass Filter of single channel

    :param X: input data
    :param Fs: sampling freq.
    :param low_cutoff: low passband
    :param high_cutoff: high passband
    :param filter_order: no of taps in FIR filter

    :return: filtered version of input data
    """
    X1 = X.reshape(-1,1)
    X1 = signal.detrend(X1,axis=0,type='constant')
    b = signal.firls(filter_order,np.array([0,low_cutoff-.1, low_cutoff, high_cutoff ,high_cutoff+.5,Fs/2]),np.array([0, 0 ,1 ,1 ,0, 0]),
                     np.array([100*0.02,0.02,0.02]),fs=Fs)
    X2 = signal.convolve(X1.reshape(-1),b,mode='same')
    return X2

def get_metadata(data,
                 wrist='left',
                 sensor_name='motionsensehrv',
                 ppg_columns=('red','infrared','green'),
                 acl_columns=('aclx','acly','aclz')):
    """
    :param data: input stream
    :param wrist: which wrist the data was collected from
    :param sensor_name: name of sensor
    :param ppg_columns: columns in the input dataframe referring to multiple ppg channels
    :param acl_columns: columns in the input dataframe referring to accelerometer channels

    :return: metadata of output stream
    """
    stream_name = "org.md2k."+str(sensor_name)+"."+str(wrist)+".wrist.bandpass.filtered"
    stream_metadata = Metadata()
    stream_metadata.set_name(stream_name).set_description("Bandpass Filtered PPG data") \
        .add_dataDescriptor(DataDescriptor().set_name("timestamp").set_type("datetime")) \
        .add_dataDescriptor(DataDescriptor().set_name("localtime").set_type("datetime")) \
        .add_dataDescriptor(DataDescriptor().set_name("version").set_type("int")) \
        .add_dataDescriptor(DataDescriptor().set_name("user").set_type("string"))

    for c in ppg_columns:
        stream_metadata.add_dataDescriptor(DataDescriptor().set_name(c).set_type("double").set_attribute("description",
                                                                                                    "ppg channel "+c))
    for c in acl_columns:
        stream_metadata.add_dataDescriptor(DataDescriptor().set_name(c).set_type("double").set_attribute("description",
                                                                                            "accelerometer channel "+c))

    stream_metadata.add_module(
        ModuleMetadata().set_name("ecg data quality").set_attribute("url", "http://md2k.org/").set_author(
            "Md Azim Ullah", "mullah@memphis.edu"))
    return stream_metadata


def bandpass_filter(
                   data,
                   Fs = 25,
                   low_cutoff = 0.4,
                   high_cutoff = 3.0,
                   filter_order = 65,
                   ppg_columns=('red','infrared','green'),
                   acl_columns=('aclx','acly','aclz'),
                   wrist='left',
                   sensor_name='motionsensehrv'):

    """

    :param data: PPG & ACL data stream
    :param Fs: sampling frequency
    :param low_cutoff: minimum frequency of pass band
    :param high_cutoff: Maximum Frequency of pass band
    :param filter_order: no. of taps in FIR filter
    :param ppg_columns: columns in the input dataframe referring to multiple ppg channels
    :param acl_columns: columns in the input dataframe referring to accelerometer channels
    :param wrist: which wrist the data was collected from
    :param sensor_name: name of sensor

    :return: Bandpass filtered version of input PPG data
    """

    ## check if all columns exist

    default_columns = ['user','version','localtime','timestamp']
    required_columns = default_columns+acl_columns+ppg_columns
    if len(set(required_columns)-set(data.columns))>0:
        raise Exception("Columns missing in input dataframe! " + str(list(set(required_columns)-set(data.columns))))

    ## select the columns from input dataframe

    data = data.select(*required_columns)

    ## udf

    default_schema = [StructField("timestamp", TimestampType()),
                      StructField("localtime", TimestampType()),
                      StructField("version", IntegerType()),
                      StructField("user", StringType())]
    schema = StructType(default_schema+[StructField(c, DoubleType()) for c in list(ppg_columns)+list(acl_columns)])
    @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
    def ppg_bandpass(data):
        data = data.sort_values('timestamp').reset_index(drop=True)
        for c in ppg_columns:
            data[c] = filter_data(data[c].values,Fs=Fs,low_cutoff=low_cutoff,high_cutoff=high_cutoff,filter_order=filter_order)
        return data

    ## steps
    ppg_bandpass_filtered = data.compute(ppg_bandpass,windowDuration=60*60*10,startTime='0 seconds')
    output_data = ppg_bandpass_filtered._data
    ds = DataStream(data=output_data,metadata=get_metadata(data,wrist=wrist,sensor_name=sensor_name,
                                                           ppg_columns=ppg_columns,acl_columns=acl_columns))
    return ds

In [ ]:
data = CC.get_stream('ppg--org.md2k.watch--fossil_watch_sport',user_id='afcfc1b5-365f-409b-918e-2f0ce8056ff9')
data  = data.withColumn('day',F.date_format('localtime',"YYYYMMdd"))
data = data.filter(F.col('day').isin(['20201020','20201019']))
# filtered_data = bandpass_filter(
#                    data,
#                    Fs = 100,
#                    low_cutoff = 0.4,
#                    high_cutoff = 3.0,
#                    filter_order = 65,
#                    ppg_columns=['ppg1'],
#                    acl_columns=[],
#                    wrist='left',
#                    sensor_name='fossil')
# CC.save_stream(filtered_data,overwrite=True)

In [ ]:
data.sort(F.col('timestamp').desc()).show(10,False)

In [ ]:
ppg = data.toPandas().sort_values('timestamp').reset_index(drop=True)

In [ ]:
data.count()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(ppg['localtime'],ppg['ppg1'])
plt.show()

In [ ]:
ppg.head()

In [ ]:
CC.list_streams()

In [ ]:
CC.get_stream('activity_type--org.md2k.moods--phonesensor--phone').sort(F.col('localtime').desc()).show(2,False)

In [ ]:
CC.list_streams()

In [ ]:
data = CC.get_stream('moods_logs').toPandas()

In [ ]:
data['ind'] = data['message'].apply(lambda a:a.find('ACTIVITY INPUT'))

In [ ]:
data[data.ind>-1][-2:]

In [ ]:
data[data.ind>-1].uuid.unique()

In [ ]:
data[(data.ind>-1)&(data.uuid=='e792585a-48a2-36ae-9a50-de0605f76829')]

In [ ]:
data = CC.get_stream('moods_logs')

In [ ]:
data  = data.withColumn('day',F.date_format('timestamp',"yyyyMMdd"))

In [ ]:
data.show(10,False)